In [8]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from langchain_community.llms import Ollama

llm = Ollama(model="mistral")

In [2]:
llm.invoke("Tell me a joke")

" Why don't scientists trust atoms?\n\nBecause they make up everything!"

In [14]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

# Simple Chain

In [20]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
output_parser = StrOutputParser()

In [8]:
chain = prompt | llm | output_parser

In [9]:
chain.invoke({"topic": "ai"})

' Why don\'t artificial intelligence ever play hide and seek with humans?\n\nBecause they always "give themselves away"! *pause for a moment to let the laughter come* AI might be good at many things, but hiding isn\'t one of them... yet!'

# More complex chain

In [1]:
from langchain_community.embeddings import OllamaEmbeddings
ollama_emb = OllamaEmbeddings(
    model="mistral",
)

In [3]:
from langchain.vectorstores import DocArrayInMemorySearch
vectorstore = DocArrayInMemorySearch.from_texts(
     ["harrison worked at kensho", "bears like to eat honey"],
     embedding = ollama_emb
)

/Users/hungryfoolish/anaconda3/envs/agent310/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [4]:
retriever = vectorstore.as_retriever()

In [12]:
retriever.get_relevant_documents("where did harrison work?")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [10]:
retriever.get_relevant_documents("what dose bears eat?")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [11]:
retriever.get_relevant_documents("harrison")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [23]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [24]:
from langchain.schema.runnable import RunnableMap
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | llm | output_parser

In [25]:
chain.invoke({"question": "where did harrison work?"})

' Harrison worked at Kensho. This information is not related to the first document about bears and honey.'

In [26]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [27]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(page_content='bears like to eat honey'),
  Document(page_content='harrison worked at kensho')],
 'question': 'where did harrison work?'}

# Bind

In [35]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

model = OllamaFunctions(model="mistral", format="json")

In [36]:
model = model.bind_tools(
    tools=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, " "e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call={"name": "get_current_weather"},
)

In [37]:
runnable = prompt | model

In [38]:
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': '{"location": "San Francisco, CA"}'}}, id='run-4f45a7ad-b7e1-475e-af8c-f10d879c96fa-0')

# Fallbacks

In [39]:
import json

In [43]:
model = Ollama(model="mistral")

In [44]:
chain = model | json.loads

In [45]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [46]:
model.invoke(challenge)

' [{"title": "Solitude", "author": "Henry David Thoreau", "firstLine": "I went to the woods because I wanted to live deliberately."},\n  { "title": "The Love Song of J. Alfred Prufrock", "author": "T.S. Eliot", "firstLine": "Let us go then, you and I, when the evening is spread out against the sky Like a patient etherized upon a table;"},\n  { "title": "Ode to a Nightingale", "author": "John Keats", "firstLine": "I heard it in the deep recesses of the night: O sweet enchantment of the still midnight! or, The Mystic Pieces of an Idyl" , "lines":[{"line":"Hail, soft-falling mask of snow!","author":"John Keats"},\n  {"line":"Is this the place where in a vision drear", "author":"John Keats"},\n  {"line":"And the cold forebodes a fearful doom:","author":"John Keats"},\n  {"line":"Fade far away, dissolve, and quite forget\\nWhat thou among the leaves hast been, what places clung,\\nThe sacred haunts of ancient joy": "author": "John Keats"}\n   ]\n}]'

In [47]:
chain.invoke(challenge)

[{'poem': {'title': 'Solitude',
   'author': 'Henry David Thoreau',
   'firstLine': 'I went to the woods because I wished to live deliberately.'}},
 {'poem': {'title': 'The Love Song of J. Alfred Prufrock',
   'author': 'T.S. Eliot',
   'firstLine': 'Let us go then, you and I, when the evening is spread out against the sky Like a patient etherized upon a table;.'}},
 {'poem': {'title': 'Annabel Lee',
   'author': 'Edgar Allan Poe',
   'firstLine': 'It was many and many a year ago, In a kingdom by the sea.'}}]

# Interface

In [48]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = Ollama(model="mistral")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [49]:
chain.invoke({"topic":  "bears"})

' Why don\'t bears like grapes?\n\nBecause they prefer ras-IN-berries! (Raspberries) This is a classic pun that plays on the homonym "rasins" (raisins) and "ras-IN-berries". Enjoy your chuckle!'

In [50]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

[' Why don\'t bears like grass?\n\nBecause they\'re not bears-y good at it! (This is a play on words. Bears aren\'t good at "bars" - which are similar to grass in sound, but also can mean being unable to do something well. But bears also don\'t like grass because they prefer their habitat in wooded areas.)',
 ' Why are frogs so happy?\n\nBecause they eat flies and have "hoppy" lives!']

In [51]:
for t in chain.stream({"topic": "bears"}):
    print(t)

 Why
 don
'
t
 bears
 like
 rain
y
 days
?




Because
 they
 are
 a
-
p
aws
ome
 in
 the
 sun
,
 but
 p
aws
ome
 dri
zz
ly
!



In [52]:
response = await chain.ainvoke({"topic": "bears"})
response

' Why don\'t bears like empty chairs?\n\nBecause they\'re used to having their "Bear-reaus"! (This one is a play on words, with "Bear-reaus" sounding like "bare asses" in English slang and "behinds" in bear terminology.) But remember, I\'m just a friendly AI, not trying to poke fun at any actual bears!'